In [1]:
# read the xcometN for community post edition
# Read the JSON file
import json

with open("output_all_spans_community.json", "r", encoding="utf-8") as f:
    loaded_data_all_spans = json.load(f)

In [2]:
loaded_data_all_spans[1]

[{'text': ['algorithme'],
  'confidence': 0.4003690779209137,
  'severity': 'minor',
  'start': 24,
  'end': 35,
  'check severity': ['minor']},
 {'text': ['grammaires', 'adjacentes', 'à', "l'arbre", 'lexicalisé', '(LTAG)'],
  'confidence': 0.3999324440956116,
  'severity': 'minor',
  'start': 53,
  'end': 103,
  'check severity': ['minor',
   'minor',
   'minor',
   'minor',
   'minor',
   'minor',
   'minor',
   'minor',
   'minor',
   'minor',
   'minor',
   'minor',
   'minor']},
 {'text': ['bilexi-cal'],
  'confidence': 0.40936070680618286,
  'severity': 'minor',
  'start': 143,
  'end': 154,
  'check severity': ['minor', 'minor', 'minor']}]

In [10]:
loaded_data_all_spans[3]

[{'text': ['linéaire', 'temporelle'],
  'confidence': 0.39508458971977234,
  'severity': 'minor',
  'start': 66,
  'end': 86,
  'check severity': ['minor', 'minor', 'minor']},
 {'text': ['longueur', "d'entrée."],
  'confidence': 0.4337560832500458,
  'severity': 'major',
  'start': 94,
  'end': 113,
  'check severity': ['major', 'major', 'minor', 'major']}]

In [11]:
one_sentence_spans = loaded_data_all_spans[3]

In [12]:
one_sentence_spans

[{'text': ['linéaire', 'temporelle'],
  'confidence': 0.39508458971977234,
  'severity': 'minor',
  'start': 66,
  'end': 86,
  'check severity': ['minor', 'minor', 'minor']},
 {'text': ['longueur', "d'entrée."],
  'confidence': 0.4337560832500458,
  'severity': 'major',
  'start': 94,
  'end': 113,
  'check severity': ['major', 'major', 'minor', 'major']}]

In [13]:
all_spans_in_this_sentence = []
all_spans_in_this_sentence_severity = []
for span in one_sentence_spans:
    all_spans_in_this_sentence.extend(span['text'].copy())
    length_of_text = len(span['text'])
    print(length_of_text)
    all_spans_in_this_sentence_severity.extend([span['severity']] * length_of_text)
print(all_spans_in_this_sentence)
print(all_spans_in_this_sentence_severity)
len(all_spans_in_this_sentence_severity)

2
2
['linéaire', 'temporelle', 'longueur', "d'entrée."]
['minor', 'minor', 'major', 'major']


4

In [3]:
loaded_data_all_spans[108]

[{'text': ['semez:'],
  'confidence': 0.4115411639213562,
  'severity': 'minor',
  'start': 25,
  'end': 32,
  'check severity': ['minor', 'minor']},
 {'text': ['des', 'risques'],
  'confidence': 0.40085238218307495,
  'severity': 'minor',
  'start': 62,
  'end': 74,
  'check severity': ['minor', 'minor']},
 {'text': ['paramètres'],
  'confidence': 0.3771682381629944,
  'severity': 'minor',
  'start': 83,
  'end': 94,
  'check severity': ['minor']}]

In [14]:
loaded_data_all_spans[121]

[{'text': ['Systèmes'],
  'confidence': 0.4589546322822571,
  'severity': 'major',
  'start': 0,
  'end': 8,
  'check severity': ['major', 'major', 'major', 'major']},
 {'text': ['traduction',
   'automatique',
   'Lium',
   'pour',
   'WMT17',
   'Nouvelles',
   'Tâche',
   'de',
   'Traduction'],
  'confidence': 0.5198729038238525,
  'severity': 'major',
  'start': 11,
  'end': 80,
  'check severity': ['major',
   'major',
   'major',
   'major',
   'major',
   'major',
   'major',
   'major',
   'major',
   'major',
   'major',
   'major',
   'major',
   'major',
   'major',
   'major',
   'major']}]

In [14]:
with open("relevant_words_in_pet_community_without_mapping.json", "r", encoding="utf-8") as f:
    relevant_words_community = json.load(f)

In [15]:
relevant_words_community[1]

['adjacentes', 'à', "l'arbre", 'lexicalisé', 'bilexi-cal']

In [16]:
print("total relevant words: ",sum(len(li) for li in relevant_words_community))

total relevant words:  2431


In [32]:
tp_community = 0
fp_community = 0
import copy

In [33]:
from difflib import SequenceMatcher
def return_opcode_results(index, words1, all_spans_in_this_sentence_severity,words2,severity_tp, severity_fp):
    matcher = SequenceMatcher(None, words1, words2)
    opcodes = matcher.get_opcodes()
    tp = 0
    fp = 0
    fn = 0
    #print("index: ", index, "opcodes", opcodes)
    #print(f"words1: {words1} and words2: {words2}")
    replace = 0
    print(f"words1: {words1}")
    print(f"words2: {words2}")
    print("opcodes: ", opcodes)
    print("it is ===============================")
    print("index is: ", index)
    
    for opcod in opcodes:
        if opcod[0] == "equal":
            tp += opcod[2] - opcod[1]
           
            all_severity = all_spans_in_this_sentence_severity[opcod[1]:opcod[2]].copy()
            for severity in all_severity:
                if severity in severity_tp:
                    severity_tp[severity] +=1
                else:
                    severity_tp[severity] =1
            print("current tp: ",  opcod[2] - opcod[1], "total tp: ", tp)
        elif opcod[0] == "delete":
            fp += opcod[2] - opcod[1]
            all_severity = all_spans_in_this_sentence_severity[opcod[1]:opcod[2]].copy()
            for severity in all_severity:
                if severity in severity_fp:
                    severity_fp[severity] +=1
                else:
                    severity_fp[severity] =1
        elif opcod[0] == "insert":
            fn += opcod[4]-opcod[3]
        else:
            replace += opcod[2] - opcod[1]
            fp += opcod[2] - opcod[1]
            all_severity = all_spans_in_this_sentence_severity[opcod[1]:opcod[2]].copy()
            for severity in all_severity:
                if severity in severity_fp:
                    severity_fp[severity] +=1
                else:
                    severity_fp[severity] =1
            fn += opcod[4] - opcod[3]
            # print("it is replaced===============================")
            
            # print(f"words1: {words1}")
            # print(f"words2: {words2}")
            # print("opcodes: ", opcodes)
        # else:
        #     fp += 
    return tp, fp, fn,replace

In [35]:
def calculate_tp_fp(all_spans, relevant_words):
    tp = 0
    fp = 0
    fn = 0
    replace = 0
    severity_tp = {}
    severity_fp = {}
    for index,each_sentence in enumerate(all_spans):
        all_spans_in_this_sentence = []
        all_spans_in_this_sentence_severity = []
        for span in each_sentence:
            all_spans_in_this_sentence.extend(span['text'].copy())
            length_of_text = len(span['text'])
            #print(length_of_text)
            all_spans_in_this_sentence_severity.extend([span['severity']] * length_of_text)
        tp_tmp, fp_tmp, fn_tmp, replace_tmp = return_opcode_results(index,all_spans_in_this_sentence,all_spans_in_this_sentence_severity, relevant_words[index],severity_tp,severity_fp)
        
        tp += tp_tmp
        fp += fp_tmp
        fn += fn_tmp
        replace += replace_tmp
        print("total tp global: ", tp)
    print("severity_tp: ", severity_tp)
    print("severity_fp: ", severity_fp)
    return tp, fp, fn, replace

In [36]:
tp, fp, fn, replace = calculate_tp_fp(loaded_data_all_spans, relevant_words_community)

words1: ['Transformation', 'de', 'structures', 'de', 'dépendances', 'en', 'arbres', 'de', 'dérivation', 'LTAG']
words2: []
opcodes:  [('delete', 0, 10, 0, 0)]
it is ===============================
index is:  0
total tp global:  0
words1: ['algorithme', 'grammaires', 'adjacentes', 'à', "l'arbre", 'lexicalisé', '(LTAG)', 'bilexi-cal']
words2: ['adjacentes', 'à', "l'arbre", 'lexicalisé', 'bilexi-cal']
opcodes:  [('delete', 0, 2, 0, 0), ('equal', 2, 6, 0, 4), ('delete', 6, 7, 4, 4), ('equal', 7, 8, 4, 5)]
it is ===============================
index is:  1
current tp:  4 total tp:  4
current tp:  1 total tp:  5
total tp global:  5
words1: ['de', 'dépendance', 'bien', 'formée', 'parseur', "d'attachement"]
words2: ['compte', 'tenu', "d'une", 'de', 'dépendance', 'formée', 'correspondante,', 'le', 'parseur', 'des', 'des']
opcodes:  [('insert', 0, 0, 0, 3), ('equal', 0, 2, 3, 5), ('delete', 2, 3, 5, 5), ('equal', 3, 4, 5, 6), ('insert', 4, 4, 6, 8), ('equal', 4, 5, 8, 9), ('replace', 5, 6, 9, 11

In [37]:
tp

925

In [38]:
fp

2050

In [39]:
fn

1506

In [40]:
replace

629

In [57]:
tp = 0
severity_tp = {}

words_in_the_spans = {'minor': ['algorithme', 'grammaires', 'adjacentes', 'à', "l'arbre", 'lexicalisé', '(LTAG)', 'bilexi-cal']}
relevant_words = ['adjacentes', 'à', "l'arbre", 'lexicalisé', 'bilexi-cal']

for severity, words in words_in_the_spans.items():
    # Iterate over a COPY of the list to avoid skipping elements
    for word in words:  # Key fix: `list(words)` creates a snapshot
        print("word: ",word)
        if word in relevant_words:
            tp += 1
            severity_tp[severity] = severity_tp.get(severity, 0) + 1
            # Remove from both lists
            words.remove(word)
            relevant_words.remove(word)

print("After removal:")
print("words_in_the_spans:", words_in_the_spans)
print("relevant_words:", relevant_words)

word:  algorithme
word:  grammaires
word:  adjacentes
word:  l'arbre
word:  (LTAG)
word:  bilexi-cal
After removal:
words_in_the_spans: {'minor': ['algorithme', 'grammaires', 'à', 'lexicalisé', '(LTAG)']}
relevant_words: ['à', 'lexicalisé']


In [56]:
tp = 0
severity_tp = {}

words_in_the_spans = {'minor': ['algorithme', 'grammaires', 'adjacentes', 'à', "l'arbre", 'lexicalisé', '(LTAG)', 'bilexi-cal']}
relevant_words = ['adjacentes', 'à', "l'arbre", 'lexicalisé', 'bilexi-cal']

for severity, words in words_in_the_spans.items():
    # Iterate over a COPY of the list to avoid skipping elements
    for word in list(words):  # Key fix: `list(words)` creates a snapshot
        if word in relevant_words:
            tp += 1
            severity_tp[severity] = severity_tp.get(severity, 0) + 1
            # Remove from both lists
            words.remove(word)
            relevant_words.remove(word)

print("After removal:")
print("words_in_the_spans:", words_in_the_spans)
print("relevant_words:", relevant_words)

After removal:
words_in_the_spans: {'minor': ['algorithme', 'grammaires', '(LTAG)']}
relevant_words: []
